In [1]:
# 딥러닝을 이용한 자연어처리
# 1. 데이터 준비
# 2. 텍스트를 표준화
# 3. 텍스트 분할(토큰화)
# 4. 어휘 인덱싱

In [2]:
import string
test_sentence = "I write, rewrite, and still rewrite again!!"
text = test_sentence.lower()
text

'i write, rewrite, and still rewrite again!!'

In [3]:
text = "".join([char for char in text if char not in string.punctuation])
text

'i write rewrite and still rewrite again'

In [4]:
# 데이터 표준화 함수
def standardize(text):
    text = text.lower()
    return "".join([char for char in text if char not in string.punctuation])

In [5]:
# 토큰화
def tokenize(text):
    return text.split()

In [6]:
# vocabulary 화
vocabulary = {"":0, "[UNK]":1}
dataset = [
    "I write, erase, rewrite",
    "Erase again, and then",
    "A poppy blooms.",
]

In [7]:
for text in dataset:
    text = standardize(text)
    tokens = tokenize(text)
    for token in tokens:
        if token not in vocabulary:
            vocabulary[token] = len(vocabulary)
dict((k,v) for k, v in vocabulary.items())

{'': 0,
 '[UNK]': 1,
 'i': 2,
 'write': 3,
 'erase': 4,
 'rewrite': 5,
 'again': 6,
 'and': 7,
 'then': 8,
 'a': 9,
 'poppy': 10,
 'blooms': 11}

In [8]:
vocabulary

{'': 0,
 '[UNK]': 1,
 'i': 2,
 'write': 3,
 'erase': 4,
 'rewrite': 5,
 'again': 6,
 'and': 7,
 'then': 8,
 'a': 9,
 'poppy': 10,
 'blooms': 11}

In [9]:
class Vectorizer:
    def standardize(self, text):
        text = text.lower()
        return "".join(char for char in text if char not in string.punctuation)
    def tokenize(self, text):
        return text.split()
    def make_vocabulary(self, dataset):
        self.vocabulary = {"":0, '[UNK]' : 1}
        for text in dataset:
            text = self.standardize(text)
            tokens = self.tokenize(text)
            for token in tokens:
                if token not in self.vocabulary:
                    self.vocabulary[token] = len(self.vocabulary)
        self.inverse_vocabulary = dict(
            (v,k) for k, v in self.vocabulary.items()
        )
    def encode(self,text):
        text = self.standardize(text)
        tokens = self.tokenize(text)
        return [self.vocabulary.get(token,1) for token in tokens]
    def decode(self, int_sequence):
        return " ".join(
            self.inverse_vocabulary.get(i,'[UNK]') for i in int_sequence
        )

In [10]:
vectorizer = Vectorizer()
dataset = [
    'I write, erase, rewrite',
    'Erase again, and then',
    'A poppy blooms'
]
vectorizer.make_vocabulary(dataset)

In [11]:
# 단어 집합에 없는 단어일 경우 UNK로 대체
vectorizer.vocabulary

{'': 0,
 '[UNK]': 1,
 'i': 2,
 'write': 3,
 'erase': 4,
 'rewrite': 5,
 'again': 6,
 'and': 7,
 'then': 8,
 'a': 9,
 'poppy': 10,
 'blooms': 11}

In [12]:
test_sentence = "I write, erase, rewrite and still rewrite again"
encoded_sentence = vectorizer.encode(test_sentence)
encoded_sentence

[2, 3, 4, 5, 7, 1, 5, 6]

In [13]:
vectorizer.inverse_vocabulary

{0: '',
 1: '[UNK]',
 2: 'i',
 3: 'write',
 4: 'erase',
 5: 'rewrite',
 6: 'again',
 7: 'and',
 8: 'then',
 9: 'a',
 10: 'poppy',
 11: 'blooms'}

In [14]:
decode_sentence = vectorizer.decode(encoded_sentence)
decode_sentence

'i write erase rewrite and [UNK] rewrite again'

In [15]:
from tensorflow.keras.layers import TextVectorization
text_vectorization = TextVectorization(output_mode='int')

C:\ProgramData\Anaconda3\lib\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.4
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [16]:
import re
import string
import tensorflow as tf

def custom_standardization_fn(string_tensor):
    lowercase_string =  tf.strings.lower(string_tensor)
    return tf.strings.regex_replace(
        lowercase_string, f"[{re.escape(string.punctuation)}]",""
    )
def custom_split_fn(string_tensor):
    return tf.strings.split(string_tensor)

text_vectorization = TextVectorization(
    output_mode='int',
    standardize=custom_standardization_fn,
    split=custom_split_fn
)

In [17]:
dataset = [
    'I write, erase, rewrite',
    'Erase again, and then',
    'A poppy blooms'
]
text_vectorization.adapt(dataset)

In [18]:
# 어휘 사전 출력
text_vectorization.get_vocabulary()

['',
 '[UNK]',
 'erase',
 'write',
 'then',
 'rewrite',
 'poppy',
 'i',
 'blooms',
 'and',
 'again',
 'a']

In [19]:
vocabulary = text_vectorization.get_vocabulary()
test_sentence = "I write, rewrite, and still rewrite again"
encode_sentence = text_vectorization(test_sentence)
encode_sentence

<tf.Tensor: shape=(7,), dtype=int64, numpy=array([ 7,  3,  5,  9,  1,  5, 10], dtype=int64)>

In [20]:
inverse_vocab = dict(enumerate(vocabulary))
decode_sentence = " ".join(inverse_vocab[int(i)] for i in encode_sentence)
decode_sentence

'i write rewrite and [UNK] rewrite again'

In [21]:
# 단어 그룹을 표현하는 두 가지 방법: 집합과 시퀀스
# IMDB 영화 리뷰 데이터 준비하기

In [22]:
# 리눅스나 코렙에서 사용가능
# !curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz
# !rm -r aclImdb/train/unsup
# !cat aclImdb/train/pos/4077_10.txt    

In [23]:
import urllib.request as req    
url = 'https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz'
filename = 'aclImdb_v1.tar.gz'
with req.urlopen(url) as f:
    with open(filename,'wb') as of:
        of.write(f.read())

In [24]:
import tarfile
with tarfile.open(filename,'r:gz') as tr:
    tr.extractall()

# train 파일에서 unsup 및 urls_unsup 파일 삭제해야 함

In [25]:
import os, pathlib, shutil, random

In [26]:
base_dir = pathlib.Path('aclImdb')
val_dir = base_dir / 'val'
train_dir = base_dir / 'train'
for category in ('neg', 'pos'):
    os.makedirs(val_dir / category)
    files = os.listdir(train_dir / category)
    random.Random(1337).shuffle(files)
    num_val_samples = int(0.2*len(files))
    val_files = files[-num_val_samples:]
    for fname in val_files:
        shutil.move(train_dir / category / fname, val_dir / category / fname)

In [27]:
from tensorflow import keras
batch_size = 32

In [30]:
train_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/train', batch_size=batch_size)
val_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/val', batch_size=batch_size)
test_ds = keras.utils.text_dataset_from_directory(
    'aclImdb/test', batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [31]:
for inputs,targets in train_ds:
    print(f"inputs.shape : {inputs.shape}")
    print(f"inputs.dtype : {inputs.dtype}")
    print(f"targets.shape : {targets.shape}")
    print(f"targets.dtype : {targets.dtype}")
    print(f"inputs[0] : {inputs[0]}")
    print(f"targets[0] : {targets[0]}")
    break;

inputs.shape : (32,)
inputs.dtype : <dtype: 'string'>
targets.shape : (32,)
targets.dtype : <dtype: 'int32'>
inputs[0] : b'I am astounded at the positive reviews for this thoroughly uninspiring film.<br /><br />Often with foreign films I skip over reviews that complain about slow pace and seeming "absence of action" as many of the best international films do not live up to the Western Hollywood model of cinematic storytelling.<br /><br />I enjoy the frequent artfulness and lack of clich\xc3\xa9 in the foreign film arena. I enjoy that many foreign films don\'t tie things up in a neat palatable little bow.<br /><br />That said, this particular film offered no redemptive value for the time I wasted watching it. No meaningful character development, no engaging story arc, no way to get emotionally involved with any of the characters on screen. <br /><br />Synopsis: A bunch of emotionally immature uptight prejudiced colonials mistreat their slaves, and a little girl gets hurt by her only fri

In [32]:
# 단어를 집합으로 처리  : BoW 방식
# TextVectorization층으로 데이터 전처리

In [33]:
text_vectorization =  TextVectorization(
    max_tokens=20000 ,
    output_mode='multi_hot'
)
text_only_train_ds =  train_ds.map(lambda x, y : x)
text_vectorization.adapt(text_only_train_ds)

In [34]:
binary_1gram_train_ds =  train_ds.map(lambda x, y : (text_vectorization(x), y),
             num_parallel_calls=4)
binary_1gram_val_ds =  val_ds.map(lambda x, y : (text_vectorization(x), y),
             num_parallel_calls=4)
binary_1gram_test_ds =  test_ds.map(lambda x, y : (text_vectorization(x), y),
             num_parallel_calls=4)

In [35]:
for inputs,targets in binary_1gram_test_ds:
    print(f"inputs.shape : {inputs.shape}")
    print(f"inputs.dtype : {inputs.dtype}")
    print(f"targets.shape : {targets.shape}")
    print(f"targets.dtype : {targets.dtype}")
    print(f"inputs[0] : {inputs[0]}")
    print(f"targets[0] : {targets[0]}")
    break;

inputs.shape : (32, 20000)
inputs.dtype : <dtype: 'float32'>
targets.shape : (32,)
targets.dtype : <dtype: 'int32'>
inputs[0] : [1. 1. 1. ... 0. 0. 0.]
targets[0] : 0


In [36]:
# 모델생성
from tensorflow import keras
from tensorflow.keras import layers
def get_model(max_tokens = 20000, hidden_dim = 16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation='sigmoid')(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1,activation = 'sigmoid')(x)
    model = keras.Model(inputs,outputs)
    model.compile(optimizer = 'rmsprop', loss = 'binary_crossentropy',metrics=['accuracy'])
    return model

In [37]:
# 이진 유니그램 모델 훈련하고 테스트
model = get_model()
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 20000)]           0         
                                                                 
 dense (Dense)               (None, 16)                320016    
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________


In [38]:
callbacks = [
    keras.callbacks.ModelCheckpoint('binary_1gram.keras', save_best_only=True)
]
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks
         )

Epoch 1/10
625/625 [==============================] - 9s 14ms/step - loss: 0.5179 - accuracy: 0.7418 - val_loss: 0.3639 - val_accuracy: 0.8646
Epoch 2/10
625/625 [==============================] - 2s 4ms/step - loss: 0.3357 - accuracy: 0.8617 - val_loss: 0.2989 - val_accuracy: 0.8792
Epoch 3/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2764 - accuracy: 0.8895 - val_loss: 0.2856 - val_accuracy: 0.8826
Epoch 4/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2495 - accuracy: 0.9036 - val_loss: 0.2820 - val_accuracy: 0.8864
Epoch 5/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2291 - accuracy: 0.9129 - val_loss: 0.2850 - val_accuracy: 0.8882
Epoch 6/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2149 - accuracy: 0.9218 - val_loss: 0.2886 - val_accuracy: 0.8886
Epoch 7/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2029 - accuracy: 0.9247 - val_loss: 0.2933 - val_accuracy: 0.8918
Epoch

In [39]:
# 이진 인코딩을 사용한 바이그램
#바이그램을 반환하는 TextVectorization 층 만들기
# 바이그램 : 문자나 음절 또는 단어인 코든 문자열에서 인접한 두 요소의 시권스

In [40]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens = 20000,
    output_mode = 'multi_hot'
)

In [41]:
text_only_train_ds =  train_ds.map(lambda x, y : x)
text_vectorization.adapt(text_only_train_ds)
binary_2gram_train_ds =  train_ds.map(lambda x, y : (text_vectorization(x), y),
             num_parallel_calls=4)
binary_2gram_val_ds =  val_ds.map(lambda x, y : (text_vectorization(x), y),
             num_parallel_calls=4)
binary_2gram_test_ds =  test_ds.map(lambda x, y : (text_vectorization(x), y),
             num_parallel_calls=4)
model = get_model()
callbacks = [
    keras.callbacks.ModelCheckpoint('binary_2gram.keras', save_best_only=True)
]
model.fit(binary_1gram_train_ds.cache(),
          validation_data=binary_1gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks
         )

Epoch 1/10
625/625 [==============================] - 9s 13ms/step - loss: 0.5139 - accuracy: 0.7463 - val_loss: 0.3605 - val_accuracy: 0.8672
Epoch 2/10
625/625 [==============================] - 3s 4ms/step - loss: 0.3329 - accuracy: 0.8643 - val_loss: 0.2973 - val_accuracy: 0.8784
Epoch 3/10
625/625 [==============================] - 3s 4ms/step - loss: 0.2745 - accuracy: 0.8910 - val_loss: 0.2827 - val_accuracy: 0.8848
Epoch 4/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2502 - accuracy: 0.9021 - val_loss: 0.2813 - val_accuracy: 0.8894
Epoch 5/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2303 - accuracy: 0.9140 - val_loss: 0.2834 - val_accuracy: 0.8904
Epoch 6/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2159 - accuracy: 0.9198 - val_loss: 0.2886 - val_accuracy: 0.8908
Epoch 7/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2051 - accuracy: 0.9241 - val_loss: 0.2935 - val_accuracy: 0.8914
Epoch

In [42]:
# TF-IDF인코딩을 사용한 바이어그램
# 토큰 카운트를 반환하는 TextVectorization 층

In [43]:
text_vectorization = TextVectorization(
    ngrams=2,
    max_tokens = 20000,
    output_mode = 'count'
)

In [44]:
# TF-IDF 가중치가 적용된 출력을 반환하는 TextBectorization 층

In [45]:
text_vectorization =  TextVectorization(
    ngrams=2,
    max_tokens=20000,
    output_mode = 'tf_idf'
)

In [46]:
# 텐서 2.8.xx 이상에서는 gup에서 오류 - 2.9에서는 해결
# with tf.device('cpu'):
#     text_vectorization.adapt(text_only_train_ds)

In [47]:
text_only_train_ds =  train_ds.map(lambda x, y : x)
text_vectorization.adapt(text_only_train_ds)
tfidf_2gram_train_ds =  train_ds.map(lambda x, y : (text_vectorization(x), y),
             num_parallel_calls=4)
tfidf_2gram_val_ds =  val_ds.map(lambda x, y : (text_vectorization(x), y),
             num_parallel_calls=4)
tfidf_2gram_test_ds =  test_ds.map(lambda x, y : (text_vectorization(x), y),
             num_parallel_calls=4)
model = get_model()
callbacks = [
    keras.callbacks.ModelCheckpoint('tfidf_2gram.keras', save_best_only=True)
]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=10,
          callbacks=callbacks
         )

Epoch 1/10
625/625 [==============================] - 9s 13ms/step - loss: 0.3955 - accuracy: 0.8295 - val_loss: 0.2643 - val_accuracy: 0.8948
Epoch 2/10
625/625 [==============================] - 2s 4ms/step - loss: 0.2331 - accuracy: 0.9169 - val_loss: 0.2495 - val_accuracy: 0.9000
Epoch 3/10
625/625 [==============================] - 2s 4ms/step - loss: 0.1887 - accuracy: 0.9355 - val_loss: 0.2541 - val_accuracy: 0.9000
Epoch 4/10
625/625 [==============================] - 2s 4ms/step - loss: 0.1626 - accuracy: 0.9453 - val_loss: 0.2686 - val_accuracy: 0.8980
Epoch 5/10
625/625 [==============================] - 2s 4ms/step - loss: 0.1484 - accuracy: 0.9510 - val_loss: 0.2786 - val_accuracy: 0.8958
Epoch 6/10
625/625 [==============================] - 2s 4ms/step - loss: 0.1292 - accuracy: 0.9578 - val_loss: 0.3010 - val_accuracy: 0.8960
Epoch 7/10
625/625 [==============================] - 2s 4ms/step - loss: 0.1224 - accuracy: 0.9609 - val_loss: 0.3052 - val_accuracy: 0.8978
Epoch

In [48]:
inputs = keras.Input(shape=(1,), dtype='string')
processed_inputs = text_vectorization(inputs)
outputs = model(processed_inputs)
inference_model = keras.Model(inputs,outputs)

In [49]:
raw_text_data = tf.convert_to_tensor([
    ['That was an excellent movie, i hate it.']    
])
raw_text_data2 = tf.convert_to_tensor([
['The movie was boring in the first half, but it got more and more interesting. Disadvantages of having a lot of CG']
])    

In [50]:
predictions = inference_model(raw_text_data)
print(f"긍정적인 리뷰일  확율 : {float(predictions[0]*100)}")

predictions = inference_model(raw_text_data2)
print(f"긍정적인 리뷰일  확율 : {float(predictions[0]*100)}")

긍정적인 리뷰일  확율 : 99.65743255615234
긍정적인 리뷰일  확율 : 20.69423484802246
